In [1]:
from Bio import Medline
from collections import defaultdict
abstract = []
source_list = []
with open("pubmed_result_medline.txt") as handle:
    for record in Medline.parse(handle):
#         source = record['SO']
#         source_list.append(source)
        for k, v in record.iteritems():
            if k == 'AB':
                abs_single = record[k]
                abstract.append(abs_single)
#             elif k == "SO":
#                 source = record[k]
#                 source_list.append(source)

abstract_dict = defaultdict(None, (enumerate(abstract)))
            


In [2]:
# This snippet pairs PMID with abstracts
rec_so = {}
with open("pubmed_result_plos_med.txt") as handle:
    for record in Medline.parse(handle):
        if 'AB' in record.keys(): 
            pmid, abstract = record['PMID'], record['AB']
            rec_so[pmid] = abstract
        
        

In [4]:
import pandas as pd
df2 = pd.DataFrame.from_dict(rec_so, orient='index').reset_index()
        

In [5]:
df2.columns = ['PMID', 'Abstract']

In [6]:
df2.head()

,PMID,Abstract
0,16584289,BACKGROUND: Streptococcus suis serotype 2 (S. ...
1,27575534,BACKGROUND: The prevention of falls among olde...
2,27116432,"In this month's editorial, the PLOS Medicine E..."
3,27923049,"BACKGROUND: Each year, over 16,000 patients di..."
4,26348035,BACKGROUND: Home-based HIV testing and counsel...


In [13]:
df2['tokenized'] = [tokenize_abstract(abstract) for abstract in df2['Abstract']]

NameError: global name 'stopwords' is not defined

In [117]:
type(df['abstract'][0])

str

## Need to find a way to pair PMID or article source ID with abstracts

In [15]:
len(abstract_dict)

6387

In [16]:
import pandas as pd
import string
import numpy as np


In [17]:
import nltk

In [18]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

##  Do I need to do this?

In [19]:
import os
import glob
import unicodedata
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import string
from nltk.tokenize import word_tokenize
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
# Lowercase, tokenize and lemmatize all the documents
def tokenize_abstract(abstract):
    clean_abstract = abstract.lower().translate(None, string.punctuation)
    tokens = word_tokenize(clean_abstract)
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in tokens]
    clean_words = [w for w in stemmed_words if w not in stopwords.words('english')]
    return ' '.join(clean_words)

In [21]:
tokenized_dict = {}
for k, v in abstract_dict.iteritems():
    token = tokenize_abstract(v)
    tokenized_dict[k] = token

In [23]:
tokenized_dict[0]

u'physiolog function epiderm cell larg determin divers morpholog flower plant special conicalshap petal epiderm cell thought influenc light captur reflect provid pollin grip molecular mechan control conic cell shape remain larg unknown develop liveconfoc imag approach quantifi geometr paramet conic cell arabidopsi thaliana athaliana genet screen identifi katanin ktn1 mutant show phenotyp decreas tip sharpen conic cell furthermor demonstr spike1 rho plant rop gtpase requir final shape format conic cell ktn1 doe livecel imag show wildtyp cell exhibit random orient cortic microtubul array earli development stage display wellord circumferenti orient microtubul array later stage contrast loss ktn1 prevent random microtubul network shift wellord array show filament actin cap typic featur sever plant epiderm cell type includ root hair leaf trichom observ growth apex conic cell dure cell develop moreov genet pharmacolog data suggest microtubul actin requir conic cell shape togeth result provid

In [24]:
df = pd.DataFrame(np.array([abstract_dict.values()]).T)

In [25]:
df.columns = ['abstract']

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [26]:
df['tokenized'] = tokenized_dict.values()

In [45]:
df.head()

,abstract,tokenized,vectorized
0,The physiological functions of epidermal cells...,physiolog function epiderm cell larg determin ...,"(0, 11435)\t0.0525712094045\n (0, 6169)\t0...."
1,Vertical growth of plants is a dynamic process...,vertic growth plant dynam process influenc gen...,"(0, 11435)\t0.0525712094045\n (0, 6169)\t0...."
2,Preventive chemotherapy has long been practice...,prevent chemotherapi long practic nematod para...,"(0, 11435)\t0.0525712094045\n (0, 6169)\t0...."
3,Traditional genome-wide scans for positive sel...,tradit genomewid scan posit select main uncov ...,"(0, 11435)\t0.0525712094045\n (0, 6169)\t0...."
4,Ribosomal DNA is one of the most variable regi...,ribosom dna one variabl region human genom res...,"(0, 11435)\t0.0525712094045\n (0, 6169)\t0...."


In [43]:
tfid = TfidfVectorizer(stop_words=stop, max_features=20000, min_df=2)
vectorized_abstracts = tfid.fit_transform(df['tokenized'])

In [44]:
df['vectorized'] = vectorized_abstracts

In [46]:
features = tfid.get_feature_names()

In [ ]:
cosine_similarity(vectorized_abstracts[0], vectorized_abstracts[1])

In [28]:
from sklearn.decomposition import NMF


In [ ]:

import sys

from Bio import Entrez

# *Always* tell NCBI who you are
Entrez.email = "vpineda@uw.edu"

def retrieve_annotation(id_list):

    """Annotates Entrez Gene IDs using Bio.Entrez, in particular epost (to
    submit the data to NCBI) and esummary to retrieve the information.
    Returns a list of dictionaries with the annotations."""

    request = Entrez.epost("gene",id=",".join(id_list))
    try:
        result = Entrez.read(request)
    except RuntimeError as e:
        #FIXME: How generate NAs instead of causing an error with invalid IDs?
        print "An error occurred while retrieving the annotations."
        print "The error returned was %s" % e
        sys.exit(-1)

    webEnv = result["WebEnv"]
    queryKey = result["QueryKey"]
    data = Entrez.esummary(db="gene", webenv=webEnv, query_key =
            queryKey)
    annotations = Entrez.read(data)

    print "Retrieved %d annotations for %d genes" % (len(annotations),
            len(id_list))

    return annotations

In [ ]:
def print_data(annotation):
    for gene_data in annotation:
        gene_id = gene_data["Id"]
        gene_symbol = gene_data["NomenclatureSymbol"]
        gene_name = gene_data["Description"]
        print "ID: %s - Gene Symbol: %s - Gene Name: %s" % (gene_id, gene_symbol, gene_name)

In [29]:
df_genes = pd.read_csv("hgnc_complete_set.txt", delimiter="\t")

/Applications/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (32,34,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [30]:
df_genes.head()

,hgnc_id,symbol,name,locus_group,locus_type,status,location,location_sortable,alias_symbol,alias_name,...,merops,imgt,iuphar,kznf_gene_catalog,mamit-trnadb,cd,lncrnadb,enzyme_id,intermediate_filament_db,rna_central_ids
0,HGNC:5,A1BG,alpha-1-B glycoprotein,protein-coding gene,gene with protein product,Approved,19q13.43,19q13.43,NaN,NaN,...,I43.950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HGNC:37133,A1BG-AS1,A1BG antisense RNA 1,non-coding RNA,"RNA, long non-coding",Approved,19q13.43,19q13.43,FLJ23569,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,URS00007E4F6E
2,HGNC:24086,A1CF,APOBEC1 complementation factor,protein-coding gene,gene with protein product,Approved,10q11.23,10q11.23,ACF|ASP|ACF64|ACF65|APOBEC1CF,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HGNC:7,A2M,alpha-2-macroglobulin,protein-coding gene,gene with protein product,Approved,12p13.31,12p13.31,FWP007|S863-7|CPAMD5,NaN,...,I39.001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HGNC:27057,A2M-AS1,A2M antisense RNA 1 (head to head),non-coding RNA,"RNA, long non-coding",Approved,12p13.31,12p13.31,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,URS00001F234A


In [31]:
df_genes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42238 entries, 0 to 42237
Data columns (total 49 columns):
hgnc_id                     42238 non-null object
symbol                      42238 non-null object
name                        42238 non-null object
locus_group                 42238 non-null object
locus_type                  42238 non-null object
status                      42238 non-null object
location                    41074 non-null object
location_sortable           41074 non-null object
alias_symbol                20794 non-null object
alias_name                  6113 non-null object
prev_symbol                 11365 non-null object
prev_name                   20703 non-null object
gene_family                 18883 non-null object
gene_family_id              18883 non-null object
date_approved_reserved      41077 non-null object
date_symbol_changed         8921 non-null object
date_name_changed           22166 non-null object
date_modified               42230 non-null 

In [36]:
df_ucsc = pd.read_csv('ucsc_downloads/gene.txt', sep='\t', header=None)

In [37]:
df_ucsc.head()

,0,1,2,3,4,5,6
0,1,Nkx2-6,NaN,NaN,XM_127732,NaN,10090
1,2,Nkx2-2,18088.0,NM_010919,U31566,NaN,10090
2,3,Nkx2-9,18094.0,NM_008701,NM_008701,NaN,10090
3,4,Hmx1,15371.0,NM_010445,NM_010445,NaN,10090
4,5,Foxo1,56458.0,NM_019739,NM_019739,NaN,10090


In [39]:
df_ucsc.columns = ['n', 'gene_name', 'locus_link', 'ref_seq_num', 'genbank', 'uniprot', 'taxon']

In [40]:
df_ucsc.head()

,n,gene_name,locus_link,ref_seq_num,genbank,uniprot,taxon
0,1,Nkx2-6,NaN,NaN,XM_127732,NaN,10090
1,2,Nkx2-2,18088.0,NM_010919,U31566,NaN,10090
2,3,Nkx2-9,18094.0,NM_008701,NM_008701,NaN,10090
3,4,Hmx1,15371.0,NM_010445,NM_010445,NaN,10090
4,5,Foxo1,56458.0,NM_019739,NM_019739,NaN,10090


In [59]:
df_ucsc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20413 entries, 0 to 20412
Data columns (total 7 columns):
n              20413 non-null int64
gene_name      20362 non-null object
locus_link     17158 non-null float64
ref_seq_num    18084 non-null object
genbank        3153 non-null object
uniprot        0 non-null float64
taxon          20413 non-null int64
dtypes: float64(2), int64(2), object(3)
memory usage: 1.1+ MB


In [78]:
gene_ucsc = set([str(name).lower() for name in df_ucsc["gene_name"] if len(str(name)) >1])

In [171]:
"large" in gene_ucsc

True

In [97]:
gene_pairs.count(True)

0

In [95]:
set_test = set("show results physiology".split())

In [98]:
set_test

{'physiology', 'results', 'show'}

In [109]:
test_text = list()
with open("test_abstract.txt") as f:
    for line in f:
        word = line.strip().lower().split()
        test_text += word

In [146]:
import string
gene_pairs = []
for abstract in df['abstract']: 
    abstract_list = abstract.split()
    for item in abstract_list:
        word = item.strip('.').strip(',').lower()
        if word in gene_ucsc:
            print word
#             pairs.append(word)
#         else:
#             pairs.append(None)
# # if pairs[0] = None:
# #     gene_pairs.append(pairs)
# # else:
# #     gene_pairs.append(None)

rho
f2
rest
pten
impact
spen
spen
spen
spen
spen
spen
cds1
pex16
pex2
brca1
brca1
brca1
myc
notch2
notch3
rb1
pten
tgfa
tgfa
tgfa
tst
tst
large
large
egfr
egfr
egfr
ii
ii
ii
ii
padi2
padi2
padi2
padi2
padi2
atm
trip13
trip13
rho
rho
rho
ph
impact
ph
large
impact
large
poln
poln
poln
poln
poln
poln
rad51
rad51
rad51c
xrcc3
rad51c
xrcc3
rad51c
xrcc3
rad51c
xrcc3
rad51
rad51
rad51
set
atg5
atg7
atg5
atg12
sirt1
sirt1
sirt1
cul3
cul3
impact
kctd5
cul3
kctd2
kctd5
mlf1
mlf1
mlf1
mlf1
camp
gal
gal
gal
large
maf
ii
rest
tlr4
tlr4
tlr4
msh6
msh6
msh6
msh6
large
dnajb1
impact
numb
dmrt1
dmrt1
dmrt1
mlh1
mlh1
mlh1
mlh3
set
impact
usp9x
usp9x
usp9x
usp9x
march7
usp9x
march7
mecp2
mecp2
mecp2
mecp2
large
mecp2
mecp2
large
set
max
myb
npy
npy
grik1
grpr
slco4c1
slco4c1
npy
grpr
slco4c1
tert
tert
large
tcf21
tcf21
tcf21
tcf21
ahr
arnt
il1a
cyp1a1
tcf21
ahr
arnt
tcf21
tcf21
ahr
ahr
tcf21
ahr
pcna
pcna
pcna
pcna
pcna
pcna
pcna
large
ii
lipe
gpnmb
g0s2
gpnmb
g0s2
lipe
psmb5
slc12a2
slc12a2
slc12a2
keap

In [183]:
def if_gene(abstract):
    gene_pairs = set()
    for item in abstract.split():
        word = item.strip('.').strip(',').lower()
        if word in gene_ucsc:
            gene_pairs.add(word)
    if len(gene_pairs) > 1:
        return " ".join(gene_pairs)
    else:
        return None

In [184]:
df['gene_pairs'] = df['abstract'].apply(if_gene)

In [185]:
df['gene_pairs_beta'] = df['tokenized'].apply(if_gene)

In [186]:
df.head()

,abstract,tokenized,vectorized,gene_pairs,gene_pairs_beta
0,The physiological functions of epidermal cells...,physiolog function epiderm cell larg determin ...,"(0, 11435)\t0.0525712094045\n (0, 6169)\t0....",None,None
1,Vertical growth of plants is a dynamic process...,vertic growth plant dynam process influenc gen...,"(0, 11435)\t0.0525712094045\n (0, 6169)\t0....",None,None
2,Preventive chemotherapy has long been practice...,prevent chemotherapi long practic nematod para...,"(0, 11435)\t0.0525712094045\n (0, 6169)\t0....",None,None
3,Traditional genome-wide scans for positive sel...,tradit genomewid scan posit select main uncov ...,"(0, 11435)\t0.0525712094045\n (0, 6169)\t0....",None,None
4,Ribosomal DNA is one of the most variable regi...,ribosom dna one variabl region human genom res...,"(0, 11435)\t0.0525712094045\n (0, 6169)\t0....",None,None


In [187]:
df.gene_pairs_beta.value_counts()

impact set                              21
ii set                                  12
impact ii                                6
ii impact                                5
impact clock                             4
hr impact                                4
ii ctcf                                  4
f2 set                                   4
impact tes                               4
set gc                                   3
ii cdk9                                  3
set rest                                 3
brca1 brca2                              3
hr rad52 rad51                           3
brca2 rad51                              3
impact pabpn1                            2
rpa1 rpa3 rpa2                           2
set ctcf                                 2
hgf met stat3                            2
morc set                                 2
ii pole                                  2
dvl2 dvl3 dvl1                           2
spo11 rad50                              2
ii cdk8    

In [155]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6387 entries, 0 to 6386
Data columns (total 4 columns):
abstract      6387 non-null object
tokenized     6387 non-null object
vectorized    6387 non-null object
gene_pairs    1521 non-null object
dtypes: object(4)
memory usage: 199.7+ KB


In [175]:
"ii" in gene_ucsc

True